In [ ]:
# THIS PROGRAM CALCULATES THE VELOCITY AND VISCOSITY PROFILES FOR LAMINAR FLOWING YIELD-POWER-LAW FLUIDS IN ECCENTRIC ANNULI• IT ALSO NUMERICALLY INTEGRATES THE VELOCITY PROFILE
# OVER AREA TO OBTAIN FLOW RATE
# BY MARK HACI 8/24/1988, LSU

In [ ]:
# In summary, the procedure for calculating the velocity distribution is as follows:
# 1. Use Eq. 3.20 and 3.21 to calculate eps_1 and eps_0
# 2. Determine delta epsilon and delta n based on the size of the grid network
# 3. Set coefficients P and Q to zero at the boundaries (eps=eps_i and eps_0)
# 4. Guess a velocity field
# 5. Calculate a viscosity field (Eq. 4.14 and so on) based on the latest velocity field
# 6. Compute coefficients Aj, Bj, Cj and Dj using Eqs. 4.18-4.21
# 7. Use recurrence equations 4.25 and 4.26 to compute coefficients P and Q from J=2 to M-1 for each column
# 8. Calculate a new velocity field employing Eq. 4.22 from J=M-1 to 2 (back substitution)
# 9. Check for convergence comparing old and new velocity fields
# 10. Repeat from step 5 until the velocity field converges.

In [ ]:
# D2 - OUTER PIPE DIAMETER, IN
# D1 - INNER PIPE DIAMETER, IN
# E - ECCENTRICITY, DLESS
# XN - FLOW BEHAVIOR INDEX, DLESS
# XK - CONSISTENCY INDEX, EQ. CP
# TAUO - YIELD POINT, LB/100 SQ• FT
# RHO - DUMMY VARIABLE, PPG
# DPDL - FRIC PRESSURE LOSS GRAD-, PSI/FT
# QFLOW - EXPECTED FLOW RATE, GAL/MIN
# NGRIDX - number of GRIDS IN ETA DITECTION
# NGRIDY - number of GRIDS IN EPSILON DIRECTION
# NITER - MAX number OF ITERATIONS
# CONLIMI- CONVERGENCE LIMIT
# CONLIM2- CONVERGENCE LIMIT
# MVELO - "1" PRINTS VELOCITY PROFILE
# MYISC - "1" PRINTS VISCOSITY PROFILE

In [46]:
import numpy as np
import pandas as pd
from math import log, sqrt, sin, cos, sinh, cosh, pi

In [36]:
# inputs

ngridx = 52
ngridy = 82

def dim():
    return np.random.normal(loc=0, scale=1, size=(ngridx, ngridy))
def arr():
    return np.random.normal(loc=0, scale=1, size=ngridy)

a = dim()
b = dim()
c = dim()
d = dim()
garea = dim()

p = dim()
q = dim()
v = dim()
vn = dim()

v1 = dim()
vis1 = dim()
vis2 = dim()
gare = dim()

visp1 = dim()
visb1 = dim()
visp2 = dim()
visb2 = dim()

delx = arr()
totl = arr()

d2 = 5
d1 = 3.5
e = 0.0001
xn = 1
xk = 0.001
tauo = 12
rho = 9.5

dpdl = 0.1
qflow = 200

niter = 1000
conlim1 = 0.001
conlim2 = 0.001

mvelo = "1"
mvisc = "1"

In [37]:
# INITIALIZING AND READING DATA

vislim = 1e-6
vguess = qflow / (2.448*(d2**2-d1**2))

if e < 0.0001:
    e = 0.0001
if e > 0.99:
    e = 0.99

zn = 1 / (2 - xn)

taud = 4.788 * tauo * ((rho/8.34)**xn * (d2*1.27)**(2*xn) / (xk/100)**2)**zn
fconst = 2262.06 * ((rho/8.34)**xn * (d2*1.27)**(2+xn) / (xk/100)**2)**zn
f = dpdl * fconst

vco = (xk/100 / (rho/8.34) / (d2*1.27)**xn)**zn
qconst = vco * (d2*1.27)**2 / 63.09
vconst = vco/30.48
vguess = vguess/vconst
viscon = (xk/100 * (rho/8.34)**(1-xn) * (d2*1.27)**(2-2*xn))**zn * 100

In [40]:
# CALCULATING THE PIPE RADIUS RATIO AND PRINT DATA

s = d1/d2

# CALCULATING THE BOUNDARIES OF THE PIPES IN BIPJLAR COORDINATES AND THE DIMENSIONS OF THE GRIDS

j1 = ngridy + 2
j2 = j1 - 1
i1 = ngridx + 2
i2 = i1 -1

total = 0
totl[1] = 0

for j in range(ngridx):
    i = ngridx - j + 1
    x8 = (i+1) / i
    x9 = ngridx + 1
    delx[j] = log(x8)**e / ((log(x9))**e**ngridx**(1-e)) * pi
    total = total + delx[j+1]

for j in range(ngridx):
    delx[j] = delx[j+1] * pi / total
    totl[j+1] = totl[j] + delx[j+1]

delx[1] = delx[2]
delx[i1] = delx[i2]
xm = e * (d2-d1)/d2
x1 = (1- s*s - xm*xm)/2/s/xm
x2 = (1- s*s - xm*xm)/2/xm
eps1 = log(x1 + sqrt(x1**2-1))
eps2 = log(x2 + sqrt(x2**2-1))
dely = (eps1-eps2) / ngridy
dsi = sinh(eps2)

In [50]:
# STARTING LINE-BY-LINE METHOD, SETTING THE COEFFIIENTS AT THE BOUNDARIES

for i in range(2, i2):
    a[i, 1] = 1
    b[i, 1] = 0
    c[i, 1] = 0
    d[i, 1] = 0
    p[i, 1] = b[i, 1] / a[i, 1]
    q[i, 1] = d[i, 1] / a[i, 1]
    p[i, j1] = 0   # TODO
    q[i, j1] = 0   # TODO


In [49]:
# GUESSING A VELOCITY FIELD

for i in range(i1):
    for j in range(j1):
        v[i, j] = vguess
        if j==1 or j==j1:
            v[i, j] = 0


In [ ]:
# INITILAZING FLOW RATE, PRINTING COUNTER, AND ITERATION COUNTER

qt1 = 0
iprint = 0
iter = 0

# REPEATING TO SETAIN AN MIMPROVES VELOCITY FIELD
# 100 CONTINUE

for i in range(2, i1):
    for j in range(2, j1):
        numer = (delx[i-1]*(v[i,j] + v[i,j-1]) + delx[i]*(v[i-1,j] + v[i-1,j-1]))
        denom = (delx[i] + delx[i-1])
        v1[i,j] = 0.5 *  numer / denom


In [51]:
# CALCULATING VISCOSITY AT THE WEST FACE OF EACH CONTROL-VOLUME

for i in range(2, i1):
    for j in range(2, j1):

        dvdx1 = abs(v[i, j] - v[i-1, j] / (delx[i-1] + delx[i]) / 2)
        dvdy1 = abs(v1[i, j] - v1[i, j+1]) / dely

        if j==2 or j==j2:
            dvdy1 = 4/3 * dvdy1
        c1 = dvdx1**2 + dvdy1**2

        if c1 < vislim:
            c1 = vislim

        eta = totl[i-1]
        eps = eps2 + (j-2)*dely + dely/2
        xi = cosh(eps) - cos(eta)
        sh1 = xi * sqrt(c1)
        visb1[i, j] = taud * dsi / abs(sh1)

        if xn <= 1:
            visp1[i, j] = abs((dsi/sh1)**(1-xn))
        else:
            visp1[i, j] = abs((sh1/dsi)**(xn-1))

        vis1[i, j] = visp1[i, j] + visb1[i, j]

In [ ]:
# CALCULATING VISCOSITY AT THE SOUTH FACE OF EACH CONTROL-VOLUME

for i in range(2, i2):
    for j in range(2, j1):

        dvdx2 = abs(v1[i, j] - v1[i+1, j]) / delx[i]
        dvdy2 = abs(v[i, j] - v[i, j-1]) / dely

        if j==2 or j==j1:
            dvdy2 = dvdy2 * 2

        c2 = dvdx2**2 + dvdy2**2

        if c2 < vislim:
            c2 = vislim

        eta = totl[i-1] + delx[i]/2
        eps = eps2 + (j-2)*dely

        if j==2:
            eps = eps + dely/4
        if j==j1:
            eps = eps - dely/4

        xi = cosh(eps) - cos(eta)
        sh2 = xi * sqrt(c2)

        visb2[i, j] = taud * dsi / abs(sh2)

        if xn <= 1:
            visp2[i, j] = abs((dsi/sh2)**(1-xn))
        else:
            visp2[i, j] = abs((sh2/dsi)**(xn-1))

        vis2[i, j] = visp2[i, j] + visb2[i, j]

In [ ]:
# COMPUTING THE COEFFICIENTS OF THE TOMA EQN.

for i in range(2, i2):
    for j in range(2, j2):

        dels = dely
        deln = dely
        dele = (delx[i] + delx[i+1]) / 2
        delw = (delx[i] + delx[i-1]) / 2

        if j==2:
            dels = dely/2
        if j==j2:
            deln = dely/2

        di = (dele + delw) / 2
        dj = (deln + dels) / 2

        an = vis2[i, j+1] * di / deln
        _as = vis2[i, j] * di / dels
        ae = vis1[i+1, j] * dj / dele

        aw = vis1[i+1,j] * dj / dele
        a[i, j] = an + _as + ae + aw
        b[i, j] = an
        c[i, j] = _as

        if i==2:
            v[i-1, j] = v[i, j]
        if i==i2:
            v[i+1, j] = v[i, j]

        eta = totl[i-1] + delx[i]/2
        eps = eps2 + (j-2)*dely + dely/2

        xi = cosh(eps) - cos(eta)
        gare[i, j] = dsi * dsi / xi / xi * di * dj

        d[i,j] = ae * v[i+1, j] + aw * v[i-1, j] + gare[i, j] * f
        p[i,j] = b[i,j] / (a[i,j] - c[i,j] * p[i,j-1])
        q[i,j] = (c[i,j] * q[i,j-1] + d[i,j]) / (a[i,j] - c[i,j] * p[i,j-1])



In [ ]:
# CALCULATING "NEW" VELOCITY FIELD

for i in range(2, i2):
    for n in range(2, j2):
        j = j2 - n + 2
        vn[i, j] = p[i, j] * v[i, j+1] + q[i, j]

for i in range(2, i2):
    for n in range(2, j2):
        v[i,j] = vn[i,j]

# COUNTING NUMBER OF ITERATIONS
# no need to code this explicitly

In [ ]:
# CALCULATINS FLOW RATE

for i in range(2, i2):
    for j in range(2, j2):

        area = area + gare[i, j]
        florate = v[i, j] * gare[i, j]
        qt = qt + florate

qt = 2 * qt
vavg = qt / 2 / area

###### convergence of the outer loop
if abs((qt1-qt)/qt < conlim1):
    # break
    pass
else:
    qt1 = qt


In [ ]:
# LIMIT ON NUMBER OF ITERATIONS

vislim = vislim * 1e-2
if vislim < 1e-50:
    pass
if abs((qtvis - qt) / qt) < conlim2:
    pass
else:
    qtvis = qt


In [48]:
# SUBROUTINE POWER

def power(xmax, ipower=0, powr=1):

    if xmax > 0.09999:
        if xmax < 1:
            pass
        else:
            xmax = 0.1 * xmax
            ipower = ipower + 1
            powr = powr / 0.1
    else:
        xmax = 10*xmax
        ipower = ipower - 1
        powr = powr * 0.1

    return xmax, ipower, powr


In [ ]:

for j in range(1, j1):
    for i in range(1, i1):
        v[i,j] = vconst * v[i,j]

jm = j1/2
vmax = v[2,jm]
vmax, _ = power(vmax)

for n in range(1, j1):
    j = j1 - n + 1

# PRINTING VISCOSITY FIELD
# ...